In [ ]:
import numpy as np
from scipy.sparse import diags, hstack, vstack, block_diag
import math

def dft_list(N, pow5):
    length = N << 3
    k = np.array([i for i in range(N)])
    return np.exp(2j * np.pi * pow5[k] / length)

def pow5_list(N):
    length = N << 3
    pow5 = np.arange(N)
    pow5[0] = 1
    for i in range(1, N):
        pow5[i] = 5 * pow5[i - 1]
        pow5[i] = pow5[i] % (length)
        # print(pow5[i])

    return pow5

N = 1 << 8
slot = N // 2

log2_value = math.log2(slot) 
depth = int(log2_value)

V_list = [None] * depth

for i in range(depth):
    block_num = 1 << i
    diag_length = slot // (1 << (i + 1))

    e_diag = np.array([1+0j for length in range(diag_length)])
    e = diags(e_diag)

    pow5 = pow5_list(diag_length)
    omiga_diag = np.array(dft_list(diag_length, pow5))
    omiga = diags(omiga_diag)

    combined_matrix0 = hstack([e, omiga])
    combined_matrix1 = hstack([e, -omiga])
    block = vstack([combined_matrix0, combined_matrix1])
    
    matrix = block_diag([block] * block_num)

    V_list[i] = matrix

    # print(matrix.toarray())

s2c_V_list = V_list # V_0

c2s_V_list = [0.5 * np.conj(V_list[i].T) for i in range(depth-1, -1, -1)] ## V_0_inv

# V = V_list[0]
# for i in range(1, depth):
#     V =  V @ V_list[i]
# print(V.toarray())

# V_inv_s2c = c2s_V_list[0]
# for i in range(1, depth):
#     # print(c2s_V_list[i].toarray())
#     V_inv_s2c =  V_inv_s2c @ c2s_V_list[i]
# print(V_inv_s2c.toarray())

# res = V*V_inv_s2c
# # print((V*V_inv_s2c).toarray())
# for i in range(0, 10):
#     print(res[i, i])
#     print(res[i, i+1])

In [15]:
length = N // 2
z = np.matrix([(i / length + 0j) for i in range(length)]).T

z_ = z
for i in range(depth-1, -1, -1):
    print('depth: ', i)
    z_ = c2s_V_list[i] @ z_
    print(z_)

V_inv_s2c = c2s_V_list[0]
for i in range(1, depth):
    # print(c2s_V_list[i].toarray())
    V_inv_s2c =  V_inv_s2c @ c2s_V_list[i]
print(V_inv_s2c @ z)

depth:  1
[[ 0.25      +0.j        ]
 [ 0.5       +0.j        ]
 [-0.23096988+0.09567086j]
 [ 0.09567086+0.23096988j]]
depth:  0
[[ 0.375     +0.j        ]
 [-0.08838835+0.08838835j]
 [-0.06764951+0.16332037j]
 [-0.16332037+0.06764951j]]
[[ 0.375     +0.j        ]
 [-0.08838835+0.08838835j]
 [-0.06764951+0.16332037j]
 [-0.16332037+0.06764951j]]


In [ ]:
## c2s then s2c

length = N // 2
z = np.matrix([(i / length + 0j) for i in range(length)]).T

V_inv = (2 / N) * np.conj(V.T)

left = V_inv @ z
right = np.conj(left)

print(left)

t0 = (1 / 2) * (left + right)
t1 = -(1 / 2) * 1j * (left - right)

z_ = V @ (t0 + 1j * t1)

print(z)
print(z_)

[[ 0.375     +0.j        ]
 [-0.08838835+0.08838835j]
 [-0.06764951+0.16332037j]
 [-0.16332037+0.06764951j]]
[[0.  +0.j]
 [0.25+0.j]
 [0.5 +0.j]
 [0.75+0.j]]
[[0.  -2.77555756e-17j]
 [0.25+4.16333634e-17j]
 [0.5 +0.00000000e+00j]
 [0.75-1.38777878e-17j]]


In [ ]:
## c2s then s2c v2

length = N // 2
z = np.matrix([(i / length + 0j) for i in range(length)]).T

left = V_inv_s2c @ z
print(left)

right = np.conj(left)

t0 = (1 / 2) * (left + right)
t1 = -(1 / 2) * 1j * (left - right)

z_ = V @ (t0 + 1j * t1)

print(z)
print(z_)

[[ 0.375     +0.j        ]
 [-0.08838835+0.08838835j]
 [-0.06764951+0.16332037j]
 [-0.16332037+0.06764951j]]
[[0.  +0.j]
 [0.25+0.j]
 [0.5 +0.j]
 [0.75+0.j]]
[[0.  -2.77555756e-17j]
 [0.25+2.77555756e-17j]
 [0.5 +2.77555756e-17j]
 [0.75-2.77555756e-17j]]


In [4]:
## s2c then c2s

length = N
t = np.matrix([(i / length + 0j) for i in range(length)]).T

t0, t1 = np.vsplit(t, 2)

z = V @ (t0 + 1j * t1)

left = V_inv @ z
right = np.conj(left)

t0 = (1 / 2) * (left + right)
t1 = -(1 / 2) * 1j * (left - right)

print(t)
print(t0)
print(t1)

[[0.   +0.j]
 [0.125+0.j]
 [0.25 +0.j]
 [0.375+0.j]
 [0.5  +0.j]
 [0.625+0.j]
 [0.75 +0.j]
 [0.875+0.j]]
[[0.   +0.j]
 [0.125+0.j]
 [0.25 +0.j]
 [0.375+0.j]]
[[0.5  +0.j]
 [0.625+0.j]
 [0.75 +0.j]
 [0.875+0.j]]


In [4]:
## dense to diag

def res_and_ext(d0, d1):
    # print('res')
    # print(d0)
    # print(d1)
    # print(np.hstack((d0, np.flip(d1))))
    return np.hstack((d0, np.flip(d1)))

def ext_diag(d0, d1):
    # print('res')
    # print(d0)
    # print(d1)
    # print(np.hstack((d0, np.flip(d1))))
    return np.hstack((d0, d1))

def is_zero_diag(d0, d1):
    l = True
    r = True
    if d0 is not None:
        l = np.all(d0 == 0)
    if d1 is not None:
        r = np.all(d1 == 0)
    return l and r

def dense_to_diag(V_list, r=2, is_res = True):
    # r = 2
    if is_res:
        rows = depth
        cols_rest = 2 * r - 1
        cols_last = r
        diags = [[0 for _ in range(cols_rest)] for _ in range(rows-1)]
        diags.extend([[0 for _ in range(cols_last)]])

        diags_index = [[0 for _ in range(cols_rest)] for _ in range(rows-1)]
        diags_index.extend([[0 for _ in range(cols_last)]])
    else:
        rows = depth
        cols_0 = r
        cols_rest = 2 * r - 1
        diags = [[0 for _ in range(cols_0)]]
        diags.extend([[0 for _ in range(cols_rest)] for _ in range(rows-1)])

        diags_index = [[0 for _ in range(cols_0)]]
        diags_index.extend([[0 for _ in range(cols_rest)] for _ in range(rows-1)])

    for i in range(depth):
        print('depth: ', i)
        V_temp = V_list[i]

        rows = V_temp.shape[0]
        index = 0
        for n in range(rows):
            if n == 0:
                diag = V_temp.diagonal()
                # print(diag)
                if not is_zero_diag(diag, None):
                    # print(index)
                    diags[i][index] = diag
                    diags_index[i][index] = n
                    print(diags_index[i][index]) 
                    print(diags[i][index]) 
                    index = index + 1
            else:
                diag0 = V_temp.diagonal(n)
                diag1 = V_temp.diagonal(n-rows)
                # print(diag0)
                # print(diag1)
                if not is_zero_diag(diag0, diag1):
                    # print(index)
                    no_zero = ext_diag(diag0, diag1)
                    diags[i][index] = no_zero
                    diags_index[i][index] = n
                    print(diags_index[i][index]) 
                    print(diags[i][index])
                    index = index + 1

    return diags, diags_index

c2s_diags, c2s_diags_index = dense_to_diag(c2s_V_list, 2, True)
s2c_diags, s2c_diags_index = dense_to_diag(s2c_V_list, 2, False)

# print("print")
# for d in range(depth):
#     print("depth: ", d)
#     for i in range(0, len(c2s_diags[d])):
#         print(c2s_diags[d][i])

depth:  0
0
[ 0.5       +0.j         -0.35355339+0.35355339j  0.5       +0.j
 -0.35355339+0.35355339j]
1
[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
3
[0.        +0.j         0.35355339-0.35355339j 0.        +0.j
 0.35355339-0.35355339j]
depth:  1
0
[ 0.5       +0.j          0.5       +0.j         -0.46193977+0.19134172j
  0.19134172+0.46193977j]
2
[ 0.5       +0.j          0.5       +0.j          0.46193977-0.19134172j
 -0.19134172-0.46193977j]
depth:  0
0
[ 1.        +0.j          1.        +0.j         -0.92387953-0.38268343j
  0.38268343-0.92387953j]
2
[ 0.92387953+0.38268343j -0.38268343+0.92387953j  1.        +0.j
  1.        +0.j        ]
depth:  1
0
[ 1.        +0.j         -0.70710678-0.70710678j  1.        +0.j
 -0.70710678-0.70710678j]
1
[0.70710678+0.70710678j 0.        +0.j         0.70710678+0.70710678j
 0.        +0.j        ]
3
[0.+0.j 1.+0.j 0.+0.j 1.+0.j]


In [17]:
## bsgs no rotate

length = N // 2
z = np.array([(i / length + 0j) for i in range(length)])
print(z)
def rotate_left(lst, k):
    if k == 0:
        return lst
    
    n = len(lst)
    k = k % n 
    return np.roll(lst, -k)

for d in range(depth-1, -1, -1):
    print('depth: ', d)
    z_tmp = z  
    # rot_index = c2s_diags_index[d][0]
    tmp = np.array(c2s_diags[d][0]) * z_tmp
    # print('diag: ', 0)
    # print(c2s_diags[d][0])
    # print('tmp: ', tmp)
    for i in range(1, len(c2s_diags[d])):
        rot_index = c2s_diags_index[d][i]
        tmp += np.array(c2s_diags[d][i]) * rotate_left(z_tmp, rot_index)
        # print('diag: ', i)
        # print('diag: ', c2s_diags[d][i])
        # print('rot: ', rot_index)
        # print('rot_z', rotate_left(z, rot_index))
        # print('tmp: ', tmp)
    z = tmp
    # print(z)
# print('result:')
print(z)

# t0 = (1 / 2) * (z + np.conj(z))
# t1 = -(1 / 2) * 1j * (z - np.conj(z))
# z = t0 + 1j * t1
# print(z)

for d in range(depth):
    z_tmp = z
    tmp = np.array(s2c_diags[d][0]) * z_tmp
    for i in range(1, len(s2c_diags[d])):
        # print(s2c_diags[d][i])
        # print(rotate_left(z, i))
        rot_index = s2c_diags_index[d][i]
        tmp += np.array(s2c_diags[d][i]) * rotate_left(z_tmp, rot_index)
    z = tmp
print(z)

[0.  +0.j 0.25+0.j 0.5 +0.j 0.75+0.j]
depth:  1
depth:  0
[ 0.375     +0.j         -0.08838835+0.08838835j -0.06764951+0.16332037j
 -0.16332037+0.06764951j]
[0.25+0.j   0.25+0.25j 0.25+0.j   0.75-0.25j]
